In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from Metrics.Python.ml_metrics import average_precision

In [2]:
def mapk_biggest(arr, y, k=5):
    bigs = np.array([])
    for p in arr:
        bigs = np.append(bigs, getBiggest(p, k))
    bigK = np.reshape(bigs,(np.shape(arr)[0], k))
    y = np.reshape(y, (len(y), 1))
    return average_precision.mapk(bigK, y, k)

In [3]:
arch = resnet34
PATH = 'data/'
sz = 64
k = 5
metrics = [mapk_biggest]

In [4]:
label_csv = 'train.csv'
n = len(list(open(label_csv)))-1
val_idxs = get_cv_idxs(n)

In [5]:
def get_data(sz):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.05)
    return ImageClassifierData.from_csv(PATH, 'train', label_csv, tfms=tfms,
                                        val_idxs=val_idxs, test_name='test')

In [6]:
sz = 64
data = get_data(sz)

In [7]:
learn = ConvLearner.pretrained(arch, data, metrics=metrics)

In [17]:
data.val_ds, data.val_y

(<fastai.dataset.FilesIndexArrayDataset at 0x14ce70ee080>,
 array([3741,  418,  651, ..., 1618, 2942,  746], dtype=int64))

In [24]:
preds = learn.TTA() # val predictions
ys = data.val_y # val y
np.shape(preds), np.shape(ys)

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 